In [ ]:
!pip install metaphor-python
!pip install fitz
!pip install pytesseract
!pip install PyMuPDF
!pip install transformers
!pip install openai
!pip install selenium
!pip install reportlab

In [ ]:
from metaphor_python import Metaphor
import pandas as pd
from metaphor_python import Metaphor
import torch
from transformers import pipeline
import spacy
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import os
import fitz  # PyMuPDF
from google.colab import files
import pytesseract
from PIL import Image
import json


In [ ]:
# Replace with your Metaphor API key
api_key = "METAPHOR-API-KEY"
metaphor = Metaphor(api_key)

In [100]:
# Define the search query and domains
query = "intern"
include_domains = ["greenhouse.io", "jobs.lever.co"]

# Perform the search
search_response = metaphor.search(query, include_domains=include_domains, use_autoprompt=False)

# Create a DataFrame to store the results

data = {
    "Serial Number": range(1, len(search_response.results) + 1),
    "ID": [result.id for result in search_response.results],
    "URL": [result.url for result in search_response.results],
    "Contents": ["" for _ in range(len(search_response.results))],
    "Keywords": ["" for _ in range(len(search_response.results))],
    "Category": ["" for _ in range(len(search_response.results))],
    "Status": ["" for _ in range(len(search_response.results))],
}

df = pd.DataFrame(data)

# Print the DataFrame
df.head(10)

,Serial Number,ID,URL,Contents,Keywords,Category,Status
0,1,qfkq0NIkt2KeQ4NnL8hP-g,https://jobs.lever.co/cohere/ed985a33-206e-425...,,,,
1,2,RbP7atmcWR4Ne0WqNcjoVA,https://jobs.lever.co/spotify/e734cd0c-bcf6-40...,,,,
2,3,VU1thlZySGG23FCy447gJw,https://jobs.lever.co/cohere/?commitment=Inter...,,,,
3,4,FJFK6McM58UaOaTL046f2A,https://jobs.lever.co/vannevarlabs-2/b39f792a-...,,,,
4,5,EanVwdQw9MzxKU0ztshrmw,https://jobs.lever.co/skdk,,,,
5,6,FzZ2uOHnPFv1wX5itILICA,https://jobs.lever.co/brooksrunning,,,,
6,7,7fYMik8iWry9DJGtjGJGig,https://jobs.lever.co/lever,,,,
7,8,GVx08KMuU9wrwOPQZk1CTQ,https://jobs.lever.co/spotify?lever-via=fZgR7T...,,,,
8,9,PMiCOb409wtgA17fRHQKXQ,https://jobs.lever.co/end,,,,
9,10,LJfzTDE7XBiBW2KUwiRfPw,https://jobs.lever.co/audius,,,,


In [102]:
# Function to fetch and populate content for a given ID
def fetch_and_populate_content(id, row_index):
    try:
        content_response = metaphor.get_contents([id])
        if content_response.contents:
            content = content_response.contents[0]
            df.at[row_index, "Contents"] = content.extract
    except Exception as e:
        print(f"Error fetching content for ID {id}: {str(e)}")

# Populate the "Contents" column using the function
for index, row in df.iterrows():
    fetch_and_populate_content(row["ID"], index)

# Print the DataFrame with the "Contents" column populated
dfcopy=df
df.head(10)

,Serial Number,ID,URL,Contents,Keywords,Category,Status
0,1,qfkq0NIkt2KeQ4NnL8hP-g,https://jobs.lever.co/cohere/ed985a33-206e-425...,<div><div><p><b>Who are we?</b></p><p>We’re a ...,,,
1,2,RbP7atmcWR4Ne0WqNcjoVA,https://jobs.lever.co/spotify/e734cd0c-bcf6-40...,<div><div><p><b>This role is closed is no long...,,,
2,3,VU1thlZySGG23FCy447gJw,https://jobs.lever.co/cohere/?commitment=Inter...,<div><div><p>No job postings match these filte...,,,
3,4,FJFK6McM58UaOaTL046f2A,https://jobs.lever.co/vannevarlabs-2/b39f792a-...,<div><div><p>About Vannevar Labs</p><p>Vanneva...,,,
4,5,EanVwdQw9MzxKU0ztshrmw,https://jobs.lever.co/skdk,<div><div><div><p>Internship</p><div><div><p><...,,,
5,6,FzZ2uOHnPFv1wX5itILICA,https://jobs.lever.co/brooksrunning,<div><div><div><p>Bothell Outlet Store</p><div...,,,
6,7,7fYMik8iWry9DJGtjGJGig,https://jobs.lever.co/lever,<div><div><p>No job postings currently open. C...,,,
7,8,GVx08KMuU9wrwOPQZk1CTQ,https://jobs.lever.co/spotify?lever-via=fZgR7T...,<div><div><div><p>Filter by:</p><div><div><p>L...,,,
8,9,PMiCOb409wtgA17fRHQKXQ,https://jobs.lever.co/end,"<div><div><h5>Associate Director, Programs</h5...",,,
9,10,LJfzTDE7XBiBW2KUwiRfPw,https://jobs.lever.co/audius,<div><div><div><p>Artist Relations</p><p>Artis...,,,


In [ ]:
'''
# Load a spaCy model (e.g., "en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

# Define a custom list of skills for demonstration
skills_list = ["Python", "Java", "SQL", "Machine Learning", "Data Analysis","Data Engineering"]

# Function to extract skills from text
def extract_skills(text):
    doc = nlp(text)
    skills = [ent.text for ent in doc.ents if ent.text in skills_list]
    return skills
# Function to extract and update skills for each row
def extract_and_update_skills(row):
    description = row["Contents"]
    extracted_skills = extract_skills(description)
    row["Keywords"] = "hello"#", ".join(extracted_skills)

# Apply the skill extraction function to each row
df=df.apply(extract_and_update_skills, axis=1)

# Print the updated DataFrame
df.head()

In [103]:
openai.api_key = 'OPENAI-API-KEY'


In [108]:
def extractSkillsOpenAi(row):
  jobdescription=row['Contents']
  USER_QUESTION = "I am sharing the entire text of a job description page for an internship, please extract out the skills mentioned in it. Write only the skills in response, separated by comma. By skills, I mean things like Python, Java, SQL, MS Office, public speaking, cooking etc. Pleae note that there might be some html tags as well, please ignore them. The job description contents is: "+jobdescription
  completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": USER_QUESTION},
      ],
  )

  return completion.choices[0].message.content
def applyall(row):
  return row['ID']
df["Keywords"]=df.apply(extractSkillsOpenAi, axis=1)
df.head(10)

,Serial Number,ID,URL,Contents,Keywords,Category,Status
0,1,qfkq0NIkt2KeQ4NnL8hP-g,https://jobs.lever.co/cohere/ed985a33-206e-425...,<div><div><p><b>Who are we?</b></p><p>We’re a ...,,,
1,2,RbP7atmcWR4Ne0WqNcjoVA,https://jobs.lever.co/spotify/e734cd0c-bcf6-40...,<div><div><p><b>This role is closed is no long...,,,
2,3,VU1thlZySGG23FCy447gJw,https://jobs.lever.co/cohere/?commitment=Inter...,<div><div><p>No job postings match these filte...,,,
3,4,FJFK6McM58UaOaTL046f2A,https://jobs.lever.co/vannevarlabs-2/b39f792a-...,<div><div><p>About Vannevar Labs</p><p>Vanneva...,,,
4,5,EanVwdQw9MzxKU0ztshrmw,https://jobs.lever.co/skdk,<div><div><div><p>Internship</p><div><div><p><...,,,
5,6,FzZ2uOHnPFv1wX5itILICA,https://jobs.lever.co/brooksrunning,<div><div><div><p>Bothell Outlet Store</p><div...,,,
6,7,7fYMik8iWry9DJGtjGJGig,https://jobs.lever.co/lever,<div><div><p>No job postings currently open. C...,,,
7,8,GVx08KMuU9wrwOPQZk1CTQ,https://jobs.lever.co/spotify?lever-via=fZgR7T...,<div><div><div><p>Filter by:</p><div><div><p>L...,,,
8,9,PMiCOb409wtgA17fRHQKXQ,https://jobs.lever.co/end,"<div><div><h5>Associate Director, Programs</h5...",,,
9,10,LJfzTDE7XBiBW2KUwiRfPw,https://jobs.lever.co/audius,<div><div><div><p>Artist Relations</p><p>Artis...,,,


In [ ]:
# skillstext=extractSkillsOpenAi(df["Contents"][3])
# print(df.Contents[3])
print(skillstext)
# this is just a demo answer

In [110]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file_path):
    text = ""

    # Open the PDF file using PyMuPDF
    pdf_document = fitz.open(pdf_file_path)

    # Loop through each page and extract text
    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        text += page.get_text()

    return text

# Function to perform OCR on an image
def perform_ocr(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

# Main function to upload a PDF file and extract text
# def uploadpdfandscan():
    # Ask the user to upload a PDF file
uploaded_files = files.upload()

# Check if any files were uploaded
if len(uploaded_files) == 0:
    print("No files were uploaded.")
    # return

# Get the first uploaded file (assuming only one file is uploaded)
pdf_file_name = list(uploaded_files.keys())[0]

# Check if the uploaded file is a PDF
if not pdf_file_name.lower().endswith('.pdf'):
    print("The uploaded file is not a PDF.")
    # return

# Extract text from the PDF
pdf_text = extract_text_from_pdf(pdf_file_name)

# Print the extracted text
print("Text extracted from the PDF:")
print(pdf_text)


# uploadpdfandscan()


Saving Sumit_Singh_Senior_Software_Engineer_2023_Shared2.pdf to Sumit_Singh_Senior_Software_Engineer_2023_Shared2.pdf
Text extracted from the PDF:
Sumit Singh - Senior Software Engineer @ HSBC
Email:thesumitsingh@gmail.com
+91 8291474891
EXPERIENCE
HSBC — Senior Software Engineer
JULY 2021 - PRESENT (>2 years)
● Leading a team of 4 developers to develop an Insurance policy purchasing platform for international customers
● Developing API microservices for insurance & payments, using Java (SpringBoot) and Node.js, deployed on Kubernetes
● Worked on frontend React app for faster processing of corporate loans
● Tech stack: React, Node.js, JavaScript, Java, Python, Spring Boot, AWS, Kubernetes, Postgres
Doubtnut — Backend Developer
FEBRUARY 2021 - JULY 2021 (6 months)
● Worked on APIs for live classes, networking features of student focussed Doubtnut app (used by approx 3 million DAU)
● Tech stack: NodeJS, Express, Redis, MySQL, MondoDB
CollegeFam — Founder & Solo Developer
MARCH 2020 - APR

In [111]:
pdf_file_name

'Sumit_Singh_Senior_Software_Engineer_2023_Shared2.pdf'

In [121]:
def extractPersonDetailsOpenAi(pdf_text):
  USER_QUESTION = "I am sharing the entire text of a person's resume. Please extract out the first name, surname, email address, phone number with country code -if available, gender. Please share the data in json format, field names are fname, surname, phone, gender, email. The contents is: "+pdf_text
  completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": USER_QUESTION},
      ],
  )

  return completion.choices[0].message.content

In [122]:
response=extractPersonDetailsOpenAi(pdf_text)

In [123]:
print(response)


{
  "fname": "Sumit",
  "surname": "Singh",
  "phone": "+91 8291474891",
  "gender": "",
  "email": "thesumitsingh@gmail.com"
}


In [124]:
jsonresp=json.loads(response)
fname=jsonresp['fname']
surname=jsonresp['surname']
phone_number=jsonresp['phone']
gender=jsonresp['gender']
resumefile=pdf_file_name

In [126]:
def generateCoverLetter(jd,resumetext):
  USER_QUESTION = "I am sharing the entire text of a person's resume and a job description. Please use that and create a cover letter. The resume text is: "+resumetext+" . \n \n The job description is:  "+jd
  completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": USER_QUESTION},
      ],
  )
  return completion.choices[0].message.content

In [128]:
jdsample="""
HSBC The Netherlands



Operating in the Netherlands since 1999, HSBC Amsterdam is one of the country’s leading international banks. We provide a range of corporate and investment banking services to Large Corporate clients. Our worldwide connections enable us to offer a global perspective to our corporate clients combined with local knowledge and expertise.


Internship Program



An internship at HSBC will allow you to build on your education and gain an invaluable insight into banking with a leading global bank. We’re looking for the future talent of our business.
The Dutch branch has an open culture of innovation and teamwork. You will be supported directly by experienced managers and encouraged to take on responsibility and bring new ideas to the table.


Key Points



Global presence: communicate and work with different countries
International team: our NL office (65 FTE) embraces more than 15 nationalities
Flat organization: opportunity to work with different product teams and learn about all parts of the business, and the possibility to (net)work with senior management
Open culture: encouraged to be proactive and present your ideas, think independently and take responsibilities
Experienced team: work and learn from experienced professionals and senior management
Rapid growth: internships seen as a pipeline for the future talent


Responsibilities



Day to day responsibilities include but are not limited to:
Develop and maintain financial models to support new business origination and review existing assets
Coordination with cross-functional teams during due diligence and deal execution phase
Assist with the preparation of internal client reviews, including
 i. Financial modeling
 ii. Financial analysis
 iii. Industry and competitor analyses
 iv. Identifying key financial and other risks
Support with prospecting of new clients and identifying further commercial cross sell opportunities on existing clients
Work in partnership with other teams to deliver the services and products of the bank in a fair and transparent manner, this includes participating in internal and external client calls and meetings
Ensure that the business meets its internal and external regulatory requirements


Relevant knowledge and skills



Student must be enrolled in a study program or educational institution for the duration of the internship
Strong command of the English language (Dutch is an added benefit)
Proactive attitude, eager to take on responsibilities and able work well in an international team
Experience with financial modeling and financial analysis
Strong writing and research skills
Excellent organizational and communication skills
Stress resilient and able to shift quickly between different responsibilities
MS Office Outlook, Word, Excel, PowerPoint proficiency
Prior internship or work experience in financial services is a plus
"""



coverresponse=generateCoverLetter(jdsample,pdf_text)

In [129]:
print(coverresponse)


Dear Hiring Manager,

I am writing to express my interest in the internship position at HSBC in the Netherlands. With a strong background as a Senior Software Engineer and a proven track record of developing innovative solutions, I believe I would be a valuable asset to your team.

In my current role as a Senior Software Engineer at HSBC, I have successfully led a team of developers in creating an Insurance policy purchasing platform for international customers. My expertise lies in developing API microservices for insurance and payments using Java (SpringBoot) and Node.js, deployed on Kubernetes. Additionally, I have worked on frontend React applications, optimizing corporate loan processing.

My experience extends beyond HSBC. As a Backend Developer at Doubtnut, I worked on APIs for live classes and networking features, reaching approximately 3 million daily active users. At CollegeFam, I founded and developed an internship job board connecting students with companies. This project g

In [131]:

def create_cover_letter(resume_text, job_description_text, output_pdf_path):
    try:
        # Create a PDF document
        doc = SimpleDocTemplate(output_pdf_path, pagesize=letter)

        # Create a list of elements to be added to the PDF
        story = []

        # Define the styles for the text
        styles = getSampleStyleSheet()
        normal_style = styles['Normal']

        # Add a title to the cover letter
        title = Paragraph("Cover Letter", styles['Title'])
        story.append(title)

        # Add the content of the letter
        story.append(Paragraph(generateCoverLetter(job_description_text,resume_text), normal_style))

        # Add a separator
        story.append(Paragraph("<hr color='black'/>", normal_style))

        # Add the job description
        story.append(Paragraph("<b>Job Description:</b>", normal_style))
        story.append(Paragraph(job_description_text, normal_style))

        # Build the PDF document
        doc.build(story)

        print(f"Cover letter created and saved as {output_pdf_path}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")



In [133]:
# Finally apply


def automate_web_form(row):
    try:
        website_url=row['URL']
        job_id=row['ID']
        jd=row['Contents']
        pdf_file_path_cover=job_id+".pdf";
        first_name=fname
        resume_text=pdf_text
        pdf_file_path='Sumit_Singh_Senior_Software_Engineer_2023_Shared2.pdf'
        # Initialize the web driver (you need to install the appropriate driver for your browser)
        driver=webdriver.Chrome()

        # Open the webpage
        driver.get(website_url)

        # Locate and fill in the form fields
        driver.find_element_by_name("first_name").send_keys(first_name)
        driver.find_element_by_name("surname").send_keys(surname)
        driver.find_element_by_name("email").send_keys(email)
        create_cover_letter(resume_text, jd, job_id)
        # Upload the PDF file
        driver.find_element_by_name("resume").send_keys(pdf_file_path)

        # Upload the cover file
        driver.find_element_by_name("cover_letter").send_keys(pdf_file_path_cover)

        # Click the submit button
        driver.find_element_by_name("submit_button").click()

        # Allow some time for the form to be submitted (you can adjust the sleep time)
        time.sleep(5)

        # Close the browser
        driver.quit()

        print("Form submission successful!")
        return "Applied"
    except Exception as e:
        print("An error occurred:", str(e))
        return "Unable to apply"






In [ ]:

df["Status"]=df.apply(automate_web_form, axis=1)

In [ ]:
df.head()